### 5-4 단순한 계층 구현하기

계산그래프의 곱셈 노드를 'MulLayer', 덧셈 노드를 'AddLayer' 라는 이름으로 구현해보자.  
여기서 계층이라 함은 신경망의 기능 단위이다.  

#### 5.4.1 곱셈 계층

모든 계층은 forward()와 backward()라는 공통의 메서드 (인터페이스)를 갖도록 구현한다. 각각 순전파와 역전파를 의미한다.  
먼저 곱셈 계층을 구현해보자.  

In [21]:
import numpy as np

class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        
        return out
    
    def backward(self, dout):
        dx = dout * self.y # x와 y를 바꾼다.
        dy = dout * self.x
        
        return dx, dy

__init__ 에서는 인스턴스 변수인 x, y를 초기화한다. 이 두 변수는 순전파 시의 입력 값을 유지하기 위해 사용한다.  
foward()에서는 x, y를 인수로 받아 두 값을 곱해서 반환한다. 반면 backward()에서는 상류에서 넘어온 미분 (dout)에 순전파 값을 서로 바꿔 곱한 후 하류로 흘려보낸다.  

In [22]:
apple = 100
apple_num = 2
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

220.00000000000003


In [23]:
# 역전파
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


#### 5.4.2 덧셈 계층

In [24]:
class AddLayer:
    def __init__(self):
        pass
    
    def forward(self, x, y):
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

덧셈 계층에서는 초기화가 필요 없으니 init에서는 pass 한다.  
foward 는 두 인수를 더해주며 backward는 상류에서 내려온 미분 dout을 그대로 하류로 흘려보낸다. 

In [25]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
mul_tax_layer = MulLayer()
add_apple_orange_layer = AddLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

# 역전파
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)



print(price)
print(dapple_num, dapple, dorange_num, dorange, dtax)

715.0000000000001
110.00000000000001 2.2 165.0 3.3000000000000003 650
